<b>Download and preprocess Sentinel-2 images</b>  
Notebook for downloading and preprocessing Sentinel-2 images from Copernicus Open Access Hub (requires account)

* Level-2A products are globally available from December 2018 onwards  
* Older images (Level-1C) in the archive are processed using a standalone Sen2Cor tool  
(http://step.esa.int/main/third-party-plugins-2/sen2cor/sen2cor_v2-8/)
* Sen2Cor-02.08.00-win64 directory path should be added to System Variables
* Images are downloaded over two different tile IDs: T19PEP (covers Bonaires) T19PFP (covers east sea of Bonaire)
* Some products are not readily available and are stored in a Long Term Archive (LTA). Running <code>download_all()</code> will trigger retrieval from LTA and make the data available within 24 hours. Unfortunately, offline products can only be requested every 30 minutes. These products were downloaded manually via Copernicus Hub.
* Sentinel products are always stored outside project directory (GitHub repository)


In [ ]:
from sentinelsat import *
from collections import OrderedDict
from datetime import datetime,timedelta, date
import pandas as pd
import getpass
import os 
import re
from glob import glob
import subprocess

<b>Downloading Sentinel-2 images</b> 

In [ ]:
#user authentication (Copernicus account)
username = getpass.getpass("Username:")
pswd = getpass.getpass("Password:")
api = SentinelAPI(username,pswd,'https://scihub.copernicus.eu/dhus')

In [ ]:
#dictionary with selected dates per tile
dates_tiles = {"T19PEP":[20180304,20180309,20180314,20180319,20190108,
                         20190128,20190212,20190304, 20190309, 20190314, 
                         20190319, 20190508, 20190513, 20190518, 20190523,
                         20190821, 20191129],
               "T19PFP":[20180304,20190304,20190428]}


In [ ]:
#retrieving product informations

products = OrderedDict()

for tile in list(dates_tiles.keys()):
    
    for d in dates_tiles[tile]:
        date = datetime.strptime(str(d),'%Y%m%d').date()
        
        #contrsuct query
        kw_query = {'platformname': 'Sentinel-2',
                    'filename':f'*_{tile}_*',
                    'date':(date, date+timedelta(days=5))} #plus 5 days to get single scene
        
        #get level-2 products if date> December 2018 
        if date>datetime.strptime(str(20181201),'%Y%m%d').date():
            kw_query['producttype']= 'S2MSI2A'
        else:
            kw_query['producttype']= 'S2MSI1C'
        
        #retrieve ID used to download the data and store to OrderedDict()      
        pp = api.query(**kw_query)
        products.update(pp)

#convert to dataframe to view product information (cloud coverage, sensing date, etc.)
df_products = api.to_dataframe(products)


In [ ]:
#store product IDs according to product type
level2_online = []
level1_online = []

#check online products
for product_id in df_products.index:
    odata = api.get_product_odata(product_id)
    print(f"{odata['title']} is available: {odata['Online']} ")
    
    #sort products
    if odata['Online'] and "MSIL2A" in odata['title']:
        level2_online.append(product_id)
    elif odata['Online'] and "MSIL1C" in odata['title']:
        level1_online.append(product_id)

In [ ]:
#create output folders for each product type
level2_dir = '...'
level1_dir = '...'
os.makedirs(level2_dir,exist_ok=True)
os.makedirs(level1_dir,exist_ok=True)

#download products to each folder
if os.path.exists(level1_dir) and os.path.exists(level2_dir):
    api.download_all(products=level1_online,directory_path=level1_dir)
    api.download_all(products=level2_online,directory_path=level2_dir)

<b>Processing level-1C to level-2A  products</b> 

In [ ]:
#set I/O directories
level2_dir = '...'
level1_dir = '...'

#get level-1C file paths
level1_files = glob(level1_dir+"/*.SAFE")

#pop-up cmd window(s) and execute Sen2Cor processor
sen2cor_dir = "../projects/Sen2Cor-02.08.00-win64"
for file in level1_files:
    cmd = f'L2A_Process --resolution 10 {file} --output_dir {level2_dir}'
    os.system(f' start cmd /k "cd {sen2cor_dir} && {cmd}" ')
